# Create New Tenant

This chains together a lot of concepts from other notebooks.

In [ ]:
pip install azure-mgmt-sql requests msal pyodbc

In [ ]:
resourceGroup = "cgmpbie"
subscriptionId = "7258b7d4-3429-4998-815b-8cd6954b7ef9"
serverName = "cgmpbiesqlserver"
tenantRoot = "cgmpbietenant"
location = "westus2"

#We use this user so that we can examine what we've created after the fact.
globalAdminUser = "chmitch@microsoftanalytics.info"

#We always add these two users, one is our service principal.  This allows us to manipulate the workspace after it's created
#the other is the app id of our service principal.  This ensures we can assign capacities to worksapces after they are created.
administrators = [globalAdminUser,'6709b293-4789-477f-aee2-607b7139e63c']

keyVault = "cgmmlservicevault"

#Add another Admin to the database for convenience so you can connect to the database in Azure Data Studio or Query Editor.
secondaryAdmin = "chmitch@microsoftanalyitcs.info"

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
from services.aadservice import AadService
credential = AadService.get_credential()

## Part 1 - Find Next Tenant Name


In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
scope = 'https://analysis.windows.net/powerbi/api/.default'
aadPBIToken = credential.get_token(scope).token

pbiApiHeaders =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadPBIToken}

In [ ]:
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups'       

apiResponse = requests.get(apiUrl, headers=pbiApiHeaders)
#error handling for API call
if apiResponse.status_code != 200:
    description = f'Error retreiving workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

In [ ]:
workspaces = apiResponse["value"]

keyPosition = 0
keyValue = 0

for workspace in workspaces:
    nameParts = workspace["name"].split(tenantRoot)
    #if there was an underscroe in the title, get the suffix of the database name.
    if len(nameParts) > 1:
        #if the suffix is greater than our max capture it as the new max
        if int(nameParts[1]) > keyValue:
            keyValue = int(nameParts[1])
keyValue = keyValue+1
tenantName = f"{tenantRoot}{keyValue}"

print(tenantName)

## Part 2 - Create Tenant Workspace

In [ ]:
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups?workspaceV2=True'       

body = {"name":tenantName}

apiResponse = requests.post(apiUrl, headers=pbiApiHeaders, data=json.dumps(body))
#error handling for createTemporaryUplodadLocation
if apiResponse.status_code != 200:
    description = f'Error creating workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

In [ ]:
workspaceId = apiResponse["id"]
print(workspaceId)

In [ ]:
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/users'       

body = {
  "emailAddress": globalAdminUser,
  "groupUserAccessRight": "Admin"
}

apiResponse = requests.post(apiUrl, headers=pbiApiHeaders, data=json.dumps(body))
#error handling for adding user
if apiResponse.status_code != 200:
    description = f'Error creating user:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    print("User added")

## Part 3:  Create Tenant Capacity

In [ ]:
scope = 'https://management.azure.com/.default'
azureMgmtToken = credential.get_token(scope).token

azureApiHeaders =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + azureMgmtToken}

In [ ]:
import requests
import json

apiUrl = f'https://management.azure.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroup}/providers/Microsoft.Fabric/capacities/{tenantName}?api-version=2022-07-01-preview'       

body = { 
            "type": "Microsoft.Fabric/capacities",
            "name": tenantName,
            "location": "westus3",
            "sku": {
                "name": "F2",
                "tier": "Fabric"
            },
            "properties": {
                "administration": {
                    "members": administrators
                }
            }
        }

apiResponse = requests.put(apiUrl, headers=azureApiHeaders, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

## Part 4:  Add Capacity to Workspace

In [ ]:
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/capacities'      

apiResponse = requests.get(apiUrl, headers=pbiApiHeaders)
#error handling for createTemporaryUplodadLocation
if apiResponse.status_code != 200:
    description = f'Error creating workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

In [ ]:
capacityId = "blank"
capacities = apiResponse["value"]
for capacity in capacities:
    if capacity["displayName"] == tenantName:
        capacityId = capacity["id"]

In [ ]:
import requests
import json

api_url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/AssignToCapacity'      

body = {'capacityId': capacityId}

api_response = requests.post(api_url, headers=pbiApiHeaders, data=json.dumps(body))

#error handling for capacity assignment.
if api_response.status_code != 200:
    description = f'Error assigning capacity:\n  -Status Code:\t{api_response.status_code}\n  -Reason:\t{api_response.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
    print(description)
else:
    print("Capacity Assigned")

## Part 5: Create Tenant Service Principal

In [ ]:
scope = 'https://graph.microsoft.com/.default'

#with the credential object, get the token for the azure management scope.
graphApiToken = credential.get_token(scope).token
graphApiHeaders =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + graphApiToken}

In [ ]:
import requests
import json

#appId = str(uuid.uuid4())
apiUrl = f'https://graph.microsoft.com/v1.0/applications'       

body = {
    "displayName": tenantName,
    "passwordCredentials":[
        {
            "displayName": "auth secret"
        }
    ]
}

apiResponse = requests.post(apiUrl, headers=graphApiHeaders, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

In [ ]:
from services.secretservice import SecretService

#Grab the app id and client secret for the created application.
appId = apiResponse["appId"]
secrets = apiResponse["passwordCredentials"]
secret = secrets[0]
secretText = secret["secretText"]

#SecretService.get_secret_byname(keyVault,"foo")
appIdKey = f'{tenantName}Id'
appSecretKey = f'{tenantName}Secret'

SecretService.store_secret_byname(keyVault, appIdKey, appId)
SecretService.store_secret_byname(keyVault, appSecretKey, secretText)

In [ ]:
import requests
import json

#appId = str(uuid.uuid4())
apiUrl = f'https://graph.microsoft.com/v1.0/servicePrincipals'       

body = {
  "appId": appId
}

apiResponse = requests.post(apiUrl, headers=graphApiHeaders, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

## Part 6:  Create Azure SQL Database

Extensive details on what this script does are covered in CreateSQLDB.ipynb

In [ ]:
from azure.mgmt.sql import SqlManagementClient
import json

try:
    #
    sqlClient = SqlManagementClient(credential=credential, subscription_id=subscriptionId)
    
    # Create database
    database = sqlClient.databases.begin_create_or_update(
        resourceGroup,
        serverName,
        tenantName,
        {
            "location": location,
            "sku": {
                "name": "S0",
                "tier": "Standard"
            }
        }
        ).result()

    print(f"Database Created:\n{database}")
except KeyError:
    print(f"Database {tenantName} create failed")


Now that we've created a database on the precreated server, we want to do some post configuration on that database.  The server's admin account is running as a service principal.  In order to let us connect to that datbabase directly we need to add an additional Entra user to the database.  I'll do this using and ODBC connection and a couple SQL commands to create a local database user that corresponds to an Entra user and also grant that user owner permissions on the database.

This is an example of where you'd likely create a tenant specific user for the database and grant that user access.

Supporting docs:

1. How to install SQL ODBC drivers on Linux:  https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline
2. How to connect to a database with pyodbc drivers and a service principal login:  https://learn.microsoft.com/en-us/azure/azure-sql/database/azure-sql-passwordless-migration-python?view=azuresql&tabs=sign-in-azure-cli%2Cazure-portal-create%2Cazure-portal-assign%2Capp-service-identity

In [ ]:
import struct
import pyodbc
import json
from services.secretservice import SecretService

serverFqdn = f'{serverName}.database.windows.net'
driver = '{ODBC Driver 18 for SQL Server}'

#Get a credential for database access.
tokenBytes = credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
token_struct = struct.pack(f'<I{len(tokenBytes)}s', len(tokenBytes), tokenBytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256

#open the connection
conn_str = f'DRIVER={driver};SERVER={serverFqdn};DATABASE={tenantName};'
conn = pyodbc.connect(conn_str,attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})

#Create an external user on the databse and grant them DBO Access
sql = f"CREATE USER [{globalAdminUser}] FROM EXTERNAL PROVIDER;"
conn.execute(sql)
sql = f"ALTER ROLE db_owner ADD MEMBER [{globalAdminUser}];"
conn.execute(sql)
sql = f"CREATE USER [{tenantName}] FROM EXTERNAL PROVIDER;"
conn.execute(sql)
sql = f"ALTER ROLE db_datareader ADD MEMBER [{tenantName}];"
conn.execute(sql)

#This is important, the connection doesn't auto commit
conn.commit()